# calculating rpkm based on formula provided [here](https://www.rna-seqblog.com/rpkm-fpkm-and-tpm-clearly-explained/)

In [12]:
import numpy as np
import pandas as pd
import scipy.stats as stats

In [13]:
df = pd.read_csv('E:\\DATA\\microglia_sequencing\\GSE99622_hanamsagar2017_raw_reads.csv')
df2 = pd.read_excel('E:\\DATA\\microglia_sequencing\\Blbo_Mgla_RPKM_calculation_example.xlsx')

In [14]:
## subsetting the secondary dataframe to just grab the columns containing transcript length info for the merge

df2 = df2[['Transcript', 'Transcript length']]

In [15]:
df

,Gene,Transcript,Best,F_E18 1,M_E18 1,F_E18 3,M_E18 4,F_P14 1,F_P14 2,F_P14 3,...,F_P60_LPS 3,M_P60_LPS 3,M_P60_LPS 4,M_P60_LPS 5,M_P60_LPS 6,M_P60_Sal 1,M_P60_Sal 2,M_P60_Sal 3,M_P60_Sal 4,F_P60_Sal 6
0,Zfp85-rs1,NM_001001130,1,7,18,9,0,4,5,1,...,6,4,1,1,11,6,19,10,17,6
1,Scap,NM_001001144,1,133,181,132,53,98,152,141,...,64,26,35,39,59,121,207,128,110,8
2,Zfp458,NM_001001152,1,20,13,14,0,5,9,4,...,14,9,6,5,9,18,22,7,10,5
3,Fbxo41,NM_001001160,1,41,15,29,7,1,3,7,...,0,0,1,1,0,2,4,0,1,0
4,Taf9b,NM_001001176,1,38,32,37,15,30,43,64,...,8,18,30,18,31,65,41,52,55,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28639,4933401P06Rik,NR_045505,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
28640,4933405E24Rik,NR_045506,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28641,4933412O06Rik,NR_045507,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
28642,4933413L06Rik,NR_045508,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df2

,Transcript,Transcript length
0,NM_001166372,4223
1,NM_001166375,3942
2,NM_175188,4391
3,NM_145486,3452
4,NM_177115,1754
...,...,...
28639,NM_001033634,10654
28640,NM_011777,2438
28641,NM_001045536,11150
28642,NM_001080755,7600


In [17]:
### merging the two dataframes by transcript column

df_merged = pd.merge(df, df2, on = 'Transcript')

In [18]:
### first we should drop any duplicate genes (only keep the uniques)

## i am doing that here by dropping rows that have a 'Best' value of 0
## then i get rid of the 'Best' and 'Transcript' columns, as they will not be helpful moving forward

for i in range(len(df)):
    if df_merged['Best'][i] == 0:
        df_merged.drop(i, axis = 0, inplace = True)
        
df_merged.drop(columns = {'Best', 'Transcript'}, inplace = True)

In [19]:
## obtaining sample indices from columns in the dataframe

samples = df_merged.columns[1:-1]

In [20]:
### create the scaling factor by dividing the total reads for each sample by 1,000,000

scaling_factor = np.zeros(shape = len(samples))
i = -1

for sample in samples:
    i = i + 1
    scaling_factor[i] = np.sum(df_merged[sample])/1000000

In [21]:
scaling_factor

array([2.280249, 1.642408, 1.911439, 1.350713, 1.732352, 2.48565 ,
       2.229777, 2.932799, 1.921692, 0.860919, 0.891773, 3.441194,
       2.975816, 2.285897, 1.849794, 2.378107, 2.737281, 1.960248,
       1.131966, 0.791256, 0.832437, 0.952973, 1.235167, 1.377098,
       1.309759, 3.118656, 0.958471, 2.539386, 0.850724, 0.865073,
       2.336264, 1.061145, 2.289659, 2.626224, 2.707992, 2.411309,
       2.064459, 2.105607, 1.024998, 0.957797, 0.959757, 0.923115,
       1.156933, 1.08747 , 2.404167, 2.516717, 2.283997, 1.889693,
       3.014192, 0.886629, 0.955904, 0.840522, 0.942251, 0.895419,
       1.310151, 2.343862, 3.176221, 1.947044, 2.165314, 0.791292])

In [22]:
samples

Index(['F_E18 1', 'M_E18 1', 'F_E18 3', 'M_E18 4', 'F_P14 1', 'F_P14 2',
       'F_P14 3', 'F_P14 4', 'F_P14 5', 'F_P14 6', 'F_P14 7', 'F_P14 8',
       'F_P14 9', 'F_P14 10', 'M_P4 1', 'F_P4 2', 'F_P4 3', 'F_P4 4',
       'F_P60_LPS 1', 'F_P60_LPS 2', 'M_P60_LPS 2', 'F_P60_LPS 4',
       'F_P60_LPS 5', 'F_P60_LPS 6', 'M_P60_LPS 7', 'F_P60_Sal 1',
       'F_P60_Sal 2', 'F_P60_Sal 3', 'F_P60_Sal 4', 'F_P60_Sal 5',
       'M_P60_Sal 5', 'F_P60_Sal 7', 'F_E18 2', 'M_E18 2', 'M_E18 3',
       'M_P14 1', 'M_P14 2', 'M_P14 3', 'M_P14 4', 'M_P14 5', 'M_P14 6',
       'M_P14 7', 'M_P14 8', 'M_P14 9', 'M_P14 10', 'F_P4 1', 'M_P4 2',
       'M_P4 3', 'M_P4 4', 'M_P60_LPS 1', 'F_P60_LPS 3', 'M_P60_LPS 3',
       'M_P60_LPS 4', 'M_P60_LPS 5', 'M_P60_LPS 6', 'M_P60_Sal 1',
       'M_P60_Sal 2', 'M_P60_Sal 3', 'M_P60_Sal 4', 'F_P60_Sal 6'],
      dtype='object')

In [23]:
### creating rpm by dividing each read count by the 'per million' scaling factor created above

rpm = df_merged.copy()
i = -1

for sample in samples:
    i = i + 1
    rpm[sample] = df_merged[sample]/scaling_factor[i]

In [24]:
rpm

,Gene,F_E18 1,M_E18 1,F_E18 3,M_E18 4,F_P14 1,F_P14 2,F_P14 3,F_P14 4,F_P14 5,...,M_P60_LPS 3,M_P60_LPS 4,M_P60_LPS 5,M_P60_LPS 6,M_P60_Sal 1,M_P60_Sal 2,M_P60_Sal 3,M_P60_Sal 4,F_P60_Sal 6,Transcript length
0,Zfp85-rs1,3.069840,10.959518,4.708494,0.000000,2.309000,2.011546,0.448475,1.704856,2.081499,...,4.758947,1.061288,1.116796,8.395979,2.559878,5.981952,5.135991,7.851055,7.582536,2218
1,Scap,58.326963,110.204042,69.057919,39.238535,56.570489,61.151007,63.235023,59.669960,57.241223,...,30.933158,37.145092,43.555028,45.032977,51.624200,65.171787,65.740682,50.800946,10.110048,4286
2,Zfp458,8.770972,7.915207,7.324325,0.000000,2.886249,3.620783,1.793901,1.704856,2.601874,...,10.707632,6.367730,5.583978,6.869437,7.679633,6.926470,3.595194,4.618268,6.318780,3488
3,Fbxo41,17.980492,9.132932,15.171816,5.182448,0.577250,1.206928,3.139327,1.022914,2.081499,...,0.000000,1.061288,1.116796,0.000000,0.853293,1.259358,0.000000,0.461827,0.000000,6562
4,Taf9b,16.664847,19.483588,19.357144,11.105246,17.317497,17.299298,28.702422,17.730502,33.303984,...,21.415263,31.838650,20.102321,23.661395,27.732008,12.908422,26.707152,25.400473,37.912680,2583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28639,4933401P06Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.763271,0.000000,0.000000,0.000000,0.000000,0.000000,1491
28640,4933405E24Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1081
28641,4933412O06Rik,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.189737,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1123
28642,4933413L06Rik,0.438549,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,922


In [25]:
### calculating RPKM by dividing the rpm values by length of transcript

rpkm = rpm[samples].div(df_merged['Transcript length']/1000, axis=0)

In [26]:
rpkm = rpkm[rpkm.all(axis = 1)]

In [27]:
age = [0] * 60
tx = [0] * 60
sex = [0] * 60

i = -1
for sample in samples:
    i = i + 1
    if 'LPS' in sample:
        tx[i] = 'LPS'
    else:
        tx[i] = 'SAL'

i = -1
for sample in samples:
    i = i + 1
    if 'M' in sample:
        sex[i] = 'Male'
    else:
        sex[i] = 'Female'
        
i = -1
for sample in samples:
    i = i + 1
    if 'E18' in sample:
        age[i] = 'E18'
    if 'P4' in sample:
        age[i] = 'P4'
    if 'P14' in sample:
        age[i] = 'P14'
    if 'P60_LPS' in sample:
        age[i] = 'P60 + LPS'
    if 'P60_SAL' in sample:
        age[i] = 'P60 + SAL'
        
for i in range(len(age)):
    if age[i] == 0:
        age[i] = 'P60'
        
age = np.array(age).astype('str')

In [28]:
rpkm.columns = [age, sex, tx, samples]

In [29]:
### reintroducing the gene column

rpkm['gene'] = df_merged['Gene']

In [30]:
rpkm.melt(id_vars = 'gene', var_name = ['age', 'sex', 'tx', 'sample'], value_name = 'expression')

,gene,age,sex,tx,sample,expression
0,Scap,E18,Female,SAL,F_E18 1,13.608718
1,Taf9b,E18,Female,SAL,F_E18 1,6.451741
2,BC031181,E18,Female,SAL,F_E18 1,56.127519
3,Baz2b,E18,Female,SAL,F_E18 1,12.365092
4,Tmem204,E18,Female,SAL,F_E18 1,11.832677
...,...,...,...,...,...,...
565855,4930592A05Rik,P60,Female,SAL,F_P60_Sal 6,40.198997
565856,Gm10336,P60,Female,SAL,F_P60_Sal 6,6.365685
565857,2810013P06Rik,P60,Female,SAL,F_P60_Sal 6,1.869462
565858,4931403E22Rik,P60,Female,SAL,F_P60_Sal 6,7.567401


In [31]:
(rpkm.melt(id_vars = 'gene', var_name = ['age', 'sex', 'tx', 'sample'], value_name = 'expression').
to_csv('C:\\Users\\Ben\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\shiny-app_200507\\GSE99622_hanamsagar2017_rpkm.csv'))

In [35]:
rpkm.columns

MultiIndex([(      'E18', 'Female', 'SAL',     'F_E18 1'),
            (      'E18',   'Male', 'SAL',     'M_E18 1'),
            (      'E18', 'Female', 'SAL',     'F_E18 3'),
            (      'E18',   'Male', 'SAL',     'M_E18 4'),
            (      'P14', 'Female', 'SAL',     'F_P14 1'),
            (      'P14', 'Female', 'SAL',     'F_P14 2'),
            (      'P14', 'Female', 'SAL',     'F_P14 3'),
            (      'P14', 'Female', 'SAL',     'F_P14 4'),
            (      'P14', 'Female', 'SAL',     'F_P14 5'),
            (      'P14', 'Female', 'SAL',     'F_P14 6'),
            (      'P14', 'Female', 'SAL',     'F_P14 7'),
            (      'P14', 'Female', 'SAL',     'F_P14 8'),
            (      'P14', 'Female', 'SAL',     'F_P14 9'),
            (      'P14', 'Female', 'SAL',    'F_P14 10'),
            (       'P4',   'Male', 'SAL',      'M_P4 1'),
            (       'P4', 'Female', 'SAL',      'F_P4 2'),
            (       'P4', 'Female', 'SAL',      'F_P4 3'

In [33]:
rpkm.set_index('gene', inplace = True)

In [34]:
rpkm.to_csv('C:\\Users\\Ben\Dropbox\\bilbo_lab_spr2020\\microglia-seq_website\\microglia-seq\\mdi_w_rpkm\\GSE99622_hanamsagar2017_rpkm_unmelted.csv')